# LLM Agents

Previously, we utilized large language models (LLMs) to solve various natural language problems. While LLMs are proficient in understanding natural language, they can make mistakes when confronted with more complex tasks. For instance, they may generate code without the ability to verify its functionality since they lack the capability to execute it. Another limitation is that these models are predominantly trained on data available before 2021, meaning they lack knowledge of current real-world events. To address this, it is necessary to provide contextual information to help them access more recent events.

One approach to achieving this is by enabling the model to perform certain actions, such as executing code, searching the internet, and utilizing other tools. The various tools are outlined in greater detail [here](https://python.langchain.com/en/latest/modules/agents/tools.html).

In the past, we employed a somewhat similar technique by combining nearest neighbors search with further rewriting answers by the LLM. However, instead of using predefined chains of tools, we now empower the LLM itself to determine which action to take in a given situation.

The second important component of the agents is the `reasoning` or `chain-of-thought` prompting (or similar).

![ReAct](https://react-lm.github.io/files/diagram.png)


Below is the example of such an approach

![ReAct Example](https://react-lm.github.io/files/hotpotqa.png)

In [ ]:
! pip install openai langchain duckduckgo-search tiktoken faiss-cpu

In [29]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI
import os

# os.environ["OPENAI_API_KEY"] = "..."

llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0)

print(llm("What model was presented by the OpenAI in November 2022?"))

/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:171: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:740: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


As an AI language model, I cannot predict future events or developments. However, as of now (September 2021), there has been no news or announcement from OpenAI regarding a model to be presented in November 2022.


In [30]:
tools = load_tools(["ddg-search", "llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

agent.run("What model was presented by the OpenAI in November 2022?")



> Entering new AgentExecutor chain...
I need to search for recent news articles about OpenAI's latest model.
Action: DuckDuckGo Search
Action Input: "OpenAI latest model November 2022"
Observation: Illustration: Ruby Chen November 30, 2022 Authors OpenAI Product, Announcements ChatGPT is a sibling model to InstructGPT, which is trained to follow an instruction in a prompt and provide a detailed response. We are excited to introduce ChatGPT to get users' feedback and learn about its strengths and weaknesses. ChatGPT is an artificial intelligence (AI) chatbot developed by OpenAI and released in November 2022. The name "ChatGPT" combines "Chat", referring to its chatbot functionality, and "GPT", which stands for Generative Pre-trained Transformer, a type of large language model (LLM). [2] Artificial intelligence ChatGPT is OpenAI's latest fix for GPT-3. It's slick but still spews nonsense The new version of the company's large language model makes stuff up—but can also admit... DALL·E 2

'OpenAI presented ChatGPT in November 2022.'

How it looks inside?

It is just a sophisticated prompt for LLM

```json
"""
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""
```

In [34]:
agent.run("What model was presented by the OpenAI in November 2022? How many tokens the model can keep in the context?")



> Entering new AgentExecutor chain...
I need to search for recent news about OpenAI's models.
Action: DuckDuckGo Search
Action Input: "OpenAI model November 2022"
Observation: November 30, 2022 Authors OpenAI Product, Announcements ChatGPT is a sibling model to InstructGPT, which is trained to follow an instruction in a prompt and provide a detailed response. We are excited to introduce ChatGPT to get users' feedback and learn about its strengths and weaknesses. During the research preview, usage of ChatGPT is free. ChatGPT is an artificial intelligence (AI) chatbot developed by OpenAI and released in November 2022. The name "ChatGPT" combines "Chat", referring to its chatbot functionality, and "GPT", which stands for Generative Pre-trained Transformer, a type of large language model (LLM). [2] Will Douglas Heaven November 30, 2022 Stephanie Arnett/MITTR Buzz around GPT-4, the anticipated but as-yet-unannounced follow-up to OpenAI's groundbreaking large language model, GPT-3, is... O

'The ChatGPT model can keep approximately 2 words per token in its context.'

It gives us the wrong answer. To improve the model's performance, we can introduce an additional planning step when the model starts, by decomposing the initial query into several steps to follow.

In [36]:
from langchain.experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner

planner = load_chat_planner(llm)
executor = load_agent_executor(llm, tools, verbose=True)
planning_agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

planning_agent.run("What model was presented by the OpenAI in November 2022? How many tokens the model can keep in the context?")



> Entering new PlanAndExecute chain...
steps=[Step(value='Research the OpenAI website and other relevant sources to find information about the model presented in November 2022.'), Step(value='Look for information about the number of tokens the model can keep in the context.'), Step(value='If the information is not readily available, look for articles or papers that discuss the model and its capabilities.'), Step(value='Once the information is found, summarize it in a clear and concise manner.'), Step(value="Given the above steps taken, respond to the user's original question.\n\n")]

> Entering new AgentExecutor chain...
Action:
```
{
  "action": "DuckDuckGo Search",
  "action_input": "OpenAI model November 2022"
}
```

Observation: November 30, 2022 Authors OpenAI Product, Announcements ChatGPT is a sibling model to InstructGPT, which is trained to follow an instruction in a prompt and provide a detailed response. We are excited to introduce ChatGPT to get users' feedback and learn 

'ChatGPT is an AI chatbot developed by OpenAI and released in November 2022. It can keep a maximum of 4,096 tokens in context and is capable of generating text that looks human-like and responding to chatbot prompts. Microsoft has also announced a $10 billion investment in the technology.'

Let's ask more comprehensive question

In [38]:
planning_agent.run("How many request are required to summarize the book with 10000 words using OpenAI ChatGPT API?")



> Entering new PlanAndExecute chain...
steps=[Step(value='Determine the average length of a response from the OpenAI ChatGPT API.'), Step(value='Divide the total number of words in the book by the average length of a response to get the approximate number of requests needed.'), Step(value="Add a buffer of 10-20% to account for any missed information or errors in the API's responses."), Step(value='Round up to the nearest whole number to get the final number of requests needed.'), Step(value="Given the above steps taken, please respond to the user's original question.\n\n")]

> Entering new AgentExecutor chain...
Action:
```
{
  "action": "DuckDuckGo Search",
  "action_input": "What is the average length of a response from the OpenAI ChatGPT API?"
}
```

Observation: This provides lower level access than the dedicated Chat Completion API, but also requires additional input validation, only supports ChatGPT (gpt-35-turbo) models, and the underlying format is more likely to change over 

ValueError: ignored

In [39]:
planning_agent.run("How many request are required to summarize the book with 10000 words using OpenAI ChatGPT API? Remember that ChatGPT API has token limit")



> Entering new PlanAndExecute chain...
steps=[Step(value='Determine the token limit of OpenAI ChatGPT API.'), Step(value='Calculate the average number of tokens per word in the book.'), Step(value='Divide the total number of tokens in the book by the token limit to get the number of requests required.'), Step(value='Round up the result from step 3 to the nearest whole number.'), Step(value="Output the result from step 4 as the answer to the user's question.\n\n")]

> Entering new AgentExecutor chain...
Action:
```
{
  "action": "DuckDuckGo Search",
  "action_input": "OpenAI ChatGPT API token limit"
}
```

Observation: The Chat Completion API is a new dedicated API for interacting with the ChatGPT and GPT-4 models. This API is the preferred method for accessing these models. It is also the only way to access the new GPT-4 models. 2 Answers Sorted by: 12 As stated in the official OpenAI article: To further explore tokenization, you can use our interactive Tokenizer tool, which allows y

'ChatGPT is an AI chatbot developed by OpenAI and released in November 2022. It can keep a maximum of 4,096 tokens in context and is capable of generating text that looks human-like and responding to chatbot prompts. Microsoft has also announced a $10 billion investment in the technology.'

Ok, now it is working a little bit better, but it forgot about initial question. To improve it we can add the long-term memory to the prompt.


In [80]:
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.agents import Tool


from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings

# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})


from langchain.experimental import AutoGPT
from langchain.chat_models import ChatOpenAI

tools = load_tools(["ddg-search", "llm-math"], llm=ChatOpenAI(temperature=0))

agent = AutoGPT.from_llm_and_tools(
    ai_name="Tom",
    ai_role="Assistant",
    tools=tools,
    llm=ChatOpenAI(temperature=0),
    memory=vectorstore.as_retriever()
)
# Set verbose to be true
agent.chain.verbose = True


agent.run(["How many requests are required to summarize the book of 10000 words using OpenAI ChatGPT API?"])



> Entering new LLMChain chain...
Prompt after formatting:
System: You are Tom, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. How many requests are required to summarize the book of 10000 words using OpenAI ChatGPT API?


Constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. DuckDuckGo Search: A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query., args

'I have completed all my objectives.'

Now, it answers properly, but forgot to round the answer

The AutoGPT pipeline has the user- and tools-memory and self-criticism. We can further extend this idea creating different roles that provides critiques to our agent (e.g. [CAMEL](https://python.langchain.com/en/latest/use_cases/agent_simulations/camel_role_playing.html)). On the other hand, we can use this pipeline to simulate the behavior of multiple agents with predefined roles (e.g. [Generative Agents](https://python.langchain.com/en/latest/use_cases/agent_simulations/characters.html))